In [1]:
import pandas
import aql
conn = aql.Connection("http://arachne.compbio.ohsu.edu")
O = conn.graph("bmeg")

In [8]:
chrom = 1
loc = 27100988
ids = []
for r in ['A', 'C', 'G', 'T']:
    for a in ['A', 'C', 'G', 'T']:
        ids.append( "variant:GRCh37:%s:%s:%s:%s:%s" % (chrom, loc, loc, r, a))
for row in O.query().V(ids):
    print row

{u'gid': u'variant:GRCh37:1:27100988:27100988:C:C', u'data': {u'end': 27100988, u'id': u'GRCh37:1:27100988:27100988:C:C,T', u'start': 27100988, u'alternateBases': u'C', u'referenceName': u'1', u'referenceGenome': u'GRCh37', u'referenceBases': u'C', u'chromosome': u'1'}, u'label': u'Variant'}
{u'gid': u'variant:GRCh37:1:27100988:27100988:C:T', u'data': {u'end': 27100988, u'id': u'GRCh37:1:27100988:27100988:C:C,T', u'start': 27100988, u'alternateBases': u'T', u'referenceName': u'1', u'referenceGenome': u'GRCh37', u'referenceBases': u'C', u'chromosome': u'1'}, u'label': u'Variant'}
